In [1]:
import random
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv("annotationRecords1.csv")

In [3]:
pred_probs_df = pd.read_csv("probs.csv")

In [4]:
#get image name from path
pred_probs_df['image_name'] = pred_probs_df['path'].apply(lambda x: x.split('/')[-1])

In [5]:
labels_df=pd.read_csv("labels.csv")

In [6]:
#get image name from path
labels_df['image_name'] = labels_df['path'].apply(lambda x: x.split('/')[-1])

In [7]:
for index, row in labels_df.iterrows():
    labels_df.at[index, :] = row.replace(-1, 0)

In [8]:
df1 = dataset[dataset['image_name'].isin(pred_probs_df['image_name'])]
df1=df1.drop(columns=['comments','_id','date','end_time', 'filepath','start_time', 'time_in_seconds','no_particular_expression',
'patientID', 'smile', 'start_time', 'time_in_seconds','wrinkled_forehead', 'unclear'])

In [9]:
#renaming columns for consistency
name_map={'au10_raising_of_upper_lip':'AU10',
       'au12_pulling_at_corner_lip':'AU12', 'au20_lip_stretcher':'AU20', 'au24_lip_presser':'AU24',
       'au25_parting_lips':'AU25', 'au26_jaw_drop':'AU26', 'au27_mouth_stretch':'AU27',
       'au43_eyes_closed':'AU43', 'au4_brow_lowering':'AU4', 'au6_cheek_raising':'AU6',
       'au7_tightning_of_eyelids':'AU7', 'au9_wrinkling_of_nose':'AU9'}
df1.rename(columns=name_map, inplace=True)

In [10]:
#grouping by image_name
grouped_df = df1.groupby(['image_name','annotator'], as_index=False).first()
grouped_df

,image_name,annotator,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43,AU4,AU6,AU7,AU9
0,2021-06-23 12-17-44_000000000593.jpg,babatundeshofolu,0,0,0,0,0,0,0,0,0,0,0,0
1,2021-06-23 12-17-44_000000000593.jpg,hannahweisman,0,0,0,0,0,0,0,0,0,0,0,0
2,2021-06-23 12-17-44_000000000593.jpg,jennifer.noa,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-06-23 12-17-44_000000000593.jpg,rishika.patel@ufl.edu\health,0,0,0,0,0,0,0,0,0,0,0,0
4,2021-06-23 12-17-44_000000000662.jpg,babatundeshofolu,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24726,2022-03-30 17-14-24_000000001763.jpg,aribahali,0,0,0,0,1,0,0,0,0,0,0,0
24727,2022-03-30 17-14-24_000000001764.jpg,aribahali,0,0,0,0,0,1,0,0,0,0,0,0
24728,2022-03-30 17-14-24_000000001765.jpg,aribahali,0,0,0,0,0,1,0,0,0,0,0,0
24729,2022-03-30 17-14-24_000000001767.jpg,aribahali,0,0,0,0,1,0,0,0,0,0,0,0


In [11]:
#drop the uncommon AU's
labels_df=labels_df.drop(columns=['dataset','path','AU1','AU2', 'AU14','AU15', 'AU17','AU23'])

In [12]:
order_mapping = {value: index for index, value in enumerate(labels_df['image_name'])}
grouped_df['order'] = grouped_df['image_name'].map(order_mapping)

In [13]:
grouped_df = grouped_df.sort_values(by='order').drop(columns='order')

In [14]:
grouped_df.columns

Index(['image_name', 'annotator', 'AU10', 'AU12', 'AU20', 'AU24', 'AU25',
       'AU26', 'AU27', 'AU43', 'AU4', 'AU6', 'AU7', 'AU9'],
      dtype='object')

In [15]:
df = grouped_df
annotators = df['annotator'].unique()
labels = df.columns[2:]

In [16]:
df.columns

Index(['image_name', 'annotator', 'AU10', 'AU12', 'AU20', 'AU24', 'AU25',
       'AU26', 'AU27', 'AU43', 'AU4', 'AU6', 'AU7', 'AU9'],
      dtype='object')

In [17]:
labels_df.columns

Index(['AU4', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU20', 'AU24', 'AU25',
       'AU26', 'AU27', 'AU43', 'image_name'],
      dtype='object')

In [20]:
df

,image_name,annotator,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43,AU4,AU6,AU7,AU9
23783,2022-01-31 23-54-30_000000000007.jpg,wkratochvil,0,0,0,0,0,0,0,1,0,0,0,0
23782,2022-01-31 23-54-30_000000000007.jpg,kmaisuria,0,0,0,0,0,0,0,1,0,0,0,0
23781,2022-01-31 23-54-30_000000000007.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
23780,2022-01-31 23-54-30_000000000007.jpg,ezequielbautista,0,0,0,0,0,0,0,1,0,0,0,0
23761,2022-01-28 05-31-32_000000000822.jpg,hannahweisman,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23755,2021-11-12 09-46-12_000000000886.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
23756,2021-11-12 09-46-12_000000000888.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
23757,2021-11-12 09-46-12_000000000890.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0
23758,2021-11-12 09-46-12_000000000893.jpg,hannahweisman,0,0,0,0,0,0,0,1,0,0,0,0


In [21]:
annotators = df['annotator'].unique()
classes = df.columns.difference(['image_name', 'annotator'])

num_samples = len(df['image_name'].unique())
num_annotators = len(annotators)
num_classes = len(classes)
labels_multiannotator = np.full((num_samples, num_annotators, num_classes), np.nan)

for i, row in df.iterrows():
    sample_idx = np.where(df['image_name'].unique() == row['image_name'])[0][0]
    annotator_idx = np.where(annotators == row['annotator'])[0][0]
    class_indices = [np.where(classes == col)[0][0] for col in classes]
    labels_multiannotator[sample_idx, annotator_idx, class_indices] = row[classes].values

In [22]:
labels_multiannotator.shape

(12549, 9, 12)

In [23]:
pred_prob = pred_probs_df[pred_probs_df['image_name'].isin(df['image_name'])]

In [24]:
pred_prob.columns

Index(['dataset', 'path', 'AU1', 'AU2', 'AU4', 'AU6', 'AU7', 'AU9', 'AU10',
       'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU24', 'AU25', 'AU26',
       'AU27', 'AU43', 'image_name'],
      dtype='object')

In [25]:
pred_prob=pred_prob.drop(columns=['dataset','path','AU1','AU2', 'AU14','AU15', 'AU17','AU23'])

In [26]:
prob=pred_prob.drop(columns=['image_name'])

In [27]:
pred_probs=prob.iloc[:, 0:].values

In [28]:
pred_probs.shape

(12549, 12)

In [33]:
labels_multiannotator

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.

In [38]:
import multiannotators
from multiannotators import get_label_quality_multiannotator

In [39]:
import imp
imp.reload(multiannotators)

<module 'multiannotators' from '/home/keerthana/omops/multiannotators.py'>

In [40]:
results=get_label_quality_multiannotator(labels_multiannotator,pred_probs,consensus_method = "majority_vote",quality_method = "crowdlab",verbose=False)

In [41]:
results["label_quality"]

,num_annotations,consensus_label,annotator_agreement,consensus_quality_score
0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.255640
1,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.045072
2,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.085338
3,4,"[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 0.75...",0.038228
4,4,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]","[1.0, 0.75, 1.0, 1.0, 0.5, 0.75, 1.0, 1.0, 0.7...",0.027882
...,...,...,...,...
12544,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.132932
12545,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.151587
12546,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.136425
12547,1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.111687


In [42]:
results["detailed_label_quality"]

,Quality_wkratochvil,Quality_kmaisuria,Quality_hannahweisman,Quality_ezequielbautista,Quality_aribahali,Quality_kaelynnrodriguez,Quality_babatundeshofolu,Quality_rishika.patel@ufl.edu\health,Quality_jennifer.noa
0,0.255640,0.255640,0.255640,0.255640,NaN,NaN,NaN,NaN,NaN
1,0.045072,0.045072,0.045072,0.045072,NaN,NaN,NaN,NaN,NaN
2,0.085338,0.085338,0.085338,0.085338,NaN,NaN,NaN,NaN,NaN
3,0.038228,0.038228,0.190705,0.164080,NaN,NaN,NaN,NaN,NaN
4,0.027882,0.026473,0.005702,0.020462,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
12544,NaN,NaN,0.132932,NaN,NaN,NaN,NaN,NaN,NaN
12545,NaN,NaN,0.151587,NaN,NaN,NaN,NaN,NaN,NaN
12546,NaN,NaN,0.136425,NaN,NaN,NaN,NaN,NaN,NaN
12547,NaN,NaN,0.111687,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
results["annotator_stats"]

,annotator_quality,agreement_with_consensus,worst_class,num_of_exampels_labelled
aribahali,0.868687,0.994890,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",848
hannahweisman,0.901496,0.952133,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",11734
babatundeshofolu,0.915509,0.915510,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3097
rishika.patel@ufl.edu\health,0.917331,0.917331,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",5647
jennifer.noa,0.939702,0.939702,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]",3321
kaelynnrodriguez,0.946970,0.946970,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",33
kmaisuria,0.972222,0.972222,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",6
ezequielbautista,0.978632,0.978632,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",39
wkratochvil,1.000000,1.000000,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",6


In [50]:
from multiannotators import get_active_learning_scores

In [51]:
import imp
imp.reload(multiannotators)

<module 'multiannotators' from '/home/keerthana/omops/multiannotators.py'>

In [52]:
active_learning_scores = get_active_learning_scores(labels_multiannotator, pred_probs,None)

<class 'pandas.core.series.Series'> 0        0.269983
1        0.876105
2        0.879187
3        0.088704
4        0.109500
           ...   
12544    0.249884
12545    0.253911
12546    0.259003
12547    0.249808
12548    0.249808
Name: consensus_quality_score, Length: 12549, dtype: float64


In [54]:
df_active_learning_scores = pd.DataFrame(active_learning_scores[0], columns=['AU4', 'AU6', 'AU7', 'AU9', 'AU10',
       'AU12', 'AU20', 'AU24', 'AU25', 'AU26',
       'AU27', 'AU43'])


In [55]:
df_active_learning_scores

,AU4,AU6,AU7,AU9,AU10,AU12,AU20,AU24,AU25,AU26,AU27,AU43
0,0.534358,0.530431,0.517498,0.501907,0.516792,0.526036,0.495150,0.506223,0.493784,0.509454,0.526042,0.527424
1,0.816412,0.834524,0.790489,0.867282,0.828055,0.818789,0.841896,0.814075,0.834650,0.858502,0.848901,0.753734
2,0.374060,0.365729,0.344706,0.374893,0.357672,0.352274,0.358600,0.358372,0.368534,0.367696,0.335644,0.357683
3,0.473126,0.480669,0.472874,0.498665,0.466019,0.480885,0.480651,0.475758,0.472161,0.486306,0.482847,0.462141
4,0.722441,0.744789,0.707358,0.779131,0.744544,0.763110,0.751534,0.752727,0.741945,0.746285,0.744040,0.715650
...,...,...,...,...,...,...,...,...,...,...,...,...
12544,0.256860,0.256840,0.247030,0.250173,0.249635,0.247195,0.254178,0.250503,0.248041,0.252720,0.248792,0.254431
12545,0.248232,0.235909,0.234143,0.245789,0.245154,0.248489,0.235710,0.236222,0.240631,0.245193,0.242370,0.247224
12546,0.016645,0.010400,0.016157,0.013413,0.012926,0.010532,0.011504,0.014760,0.013430,0.012463,0.011932,0.011708
12547,0.673185,0.648976,0.588400,0.691942,0.686636,0.661372,0.703944,0.689594,0.644150,0.589028,0.657560,0.638400
